<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/rf/examples/diffusion_foldcond.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title setup **RFdiffusion** (~1min)
%%time
import os, time
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/60f09a193fb5e5ccdc4980417708dbab/Complex_Fold_base_ckpt.pt; \
  touch params/done.txt \
  ) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip -q install jedi omegaconf hydra-core icecream py3Dmol")
  os.system("pip -q install dgl -f https://data.dgl.ai/wheels/cu117/repo.html")
  os.system("cd RFdiffusion/env/SE3Transformer; pip -q install --no-cache-dir -r requirements.txt; pip -q install .")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Complex_Fold_base_ckpt.pt"]
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")

In [ ]:
#@title run **RFdiffusion** to generate a backbone {run: "auto"}
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import GridBox, Button, BoundedIntText, Output, Label, Layout, HBox, VBox
import numpy as np
import torch
import sys, os, random, string
import matplotlib.pyplot as plt
if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

name = "test"
elements = 5 #@param ["2", "3", "4", "5", "6", "7", "8", "9", "10"] {type:"raw"}
#iterations = 20 #@param ["20", "40", "80"] {type:"raw"}

global path
path = name
os.makedirs(f"outputs/{path}", exist_ok=True)

import os
import re
import sys
import time
import subprocess
from IPython.display import display
from ipywidgets import FloatProgress
def run_command_and_monitor_progress(command, trigger, total_timesteps=49):
    # Create a progress bar
    progress = FloatProgress(min=0, max=1, description='running', bar_style='info')
    display(progress)

    # Define a regular expression pattern to match lines containing the trigger
    pattern = re.compile(f'.*{trigger}.*')

    # Initialize progress_counter and total_timesteps
    progress_counter = 0

    # Run the command and get its output
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)

    while True:
        # Read one line from the output
        line = process.stdout.readline()

        # If the line is empty, the process has terminated
        if not line: break
        
        # If the line contains the trigger, update the progress bar
        if pattern.match(line):
            progress_counter += 1
            progress.value = progress_counter / total_timesteps

    # Wait for the process to terminate and get its return code
    return_code = process.wait()

    # Check for errors
    if return_code != 0:
        print('Error:', process.stderr.read())

    progress.description = "done"

def on_click(button):
  i, j = button.row, button.col
  if i == j:
    button.description = 'H' if button.description == 'E' else 'E'
    button.style.button_color = 'red' if button.description == 'H' else 'yellow'
    length = buttons[i][-1]
    length.value = 19 if button.description == 'H' else 5
  else:
    button.description = '0' if button.description == '1' else '1'
    button.style.button_color = 'darkgray' if button.style.button_color == 'white' else 'white'
    symmetric_button = buttons[j][i]
    symmetric_button.style.button_color = button.style.button_color
    symmetric_button.description = button.description

buttons = []
for i in range(elements):
  row = []
  for j in range(elements):
    button = Button(description='H' if i == j else '0',
                    layout=widgets.Layout(width='40px', height='40px', border='2px solid black'))
    button.row, button.col = i, j
    if i == j:
      button.style.button_color = 'red'
    else:
      button.style.button_color = 'white'
    button.on_click(on_click)
    row.append(button)
  button = BoundedIntText(value=19, min=2, max=30,
                          layout=widgets.Layout(width='50px'))
  button.row, button.col = i, elements
  row.append(button)
  buttons.append(row)


button_grid = GridBox([btn for row in buttons for btn in row],
                      layout=widgets.Layout(grid_template_columns=f"repeat({elements+1}, 42px)",
                                            grid_template_rows=f"repeat({elements}, 42px)",
                                            grid_gap="2px"))

loop_length = BoundedIntText(description='loop_length', value=5, min=1, max=30)
mask_loops = widgets.Checkbox(value=True, description='mask_loops', disabled=False)

def reset(button):
  for i in range(elements):
    for j in range(elements):
      b = buttons[i][j]
      if i == j:
        b.style.button_color = 'red'
        b.description = 'H'
      else:
        b.style.button_color = 'white'
        b.description = '0'
    b = buttons[i][-1]
    b.value = 19
  loop_length.value = 5
  mask_loops.value = True

def save_matrix(button):
  global path
  while os.path.exists(f"outputs/{path}_0.pdb"):
    path = name + "_" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
    os.makedirs(f"outputs/{path}", exist_ok=True)

  ########################
  loop = loop_length.value
  sse_L = [rows[-1].value for rows in buttons]
  L = (elements + 1) * loop + sum(sse_L)
  adj = np.zeros((L,L))
  sse = np.full((L,),2)
  n = loop
  for i in range(elements):
    ss = {"H":0,"E":1,"C":2}[buttons[i][i].description]
    sse[n:n+sse_L[i]] = ss
    m = loop
    for j in range(elements):
      if i != j:
        val = int(buttons[i][j].description)
        adj[n:n+sse_L[i],m:m+sse_L[j]] = val
      m += sse_L[j] + loop
    n += sse_L[i] + loop
  adj += adj.T
  ########################
  torch.save(torch.from_numpy(sse).float(),f"outputs/{path}/tmp_ss.pt")
  torch.save(torch.from_numpy(adj).float(),f"outputs/{path}/tmp_adj.pt")
  with output:
    output.clear_output()
    cmd = ["./RFdiffusion/run_inference.py",
           "inference.num_designs=1",
           f"inference.output_prefix=outputs/{path}",
           "scaffoldguided.scaffoldguided=True",
           "scaffoldguided.target_pdb=False",
           f"scaffoldguided.mask_loops={mask_loops.value}",
           f"scaffoldguided.scaffold_dir=outputs/{path}"]
    cmd_str = " ".join(cmd)
    run_command_and_monitor_progress(cmd_str, "Timestep")

reset_button = Button(description='reset', layout=widgets.Layout(width='84px', height='40px', border='2px solid black'))
reset_button.on_click(reset)

diffuse_button = Button(description='diffuse', layout=widgets.Layout(width='84px', height='40px', border='2px solid black'))
diffuse_button.on_click(save_matrix)

output = Output()
display(
    VBox([
        button_grid,
        widgets.HTML(value="<b>Options</b>"),
        loop_length,
        mask_loops,
        HBox([diffuse_button, reset_button])]))
display(output)

In [ ]:
#@title Display 3D structure {run: "auto"}
import py3Dmol

from string import ascii_uppercase,ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)

show_mainchains = False 
animate = False #@param {type:"boolean"}
hbondCutoff = 4.0
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')

if animate:
  pdb = f"/content/outputs/traj/{path}_0_pX0_traj.pdb"
  pdb_str = open(pdb,'r').read()
  view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
else:
  pdb = f"/content/outputs/{path}_0.pdb"
  pdb_str = open(pdb,'r').read()
  view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})

  view.setStyle({'cartoon': {'color':'spectrum'}})

view.zoomTo()
if animate:
  view.animate({'loop': 'backAndForth'})
view.show()